In [ ]:
import numpy as np
import pandas as pd 
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split
from PIL import Image
from keras.optimizers import Adam

In [ ]:
img_width = 150
img_height = 150
TRAIN_DIR = '/home/Addy/MURA-v1.1/training/'
#TEST_DIR = '/home/Addy/MURA-v1.1/testing/'
MURA_train = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]
#MURA_test = [TEST_DIR+i for i in os.listdir(TEST_DIR)]

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

In [ ]:
def prepare_data(list_of_images):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """
    x = [] # images as arrays
    y = [] # labels
    
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (img_width,img_height)))#, interpolation=cv2.INTER_CUBIC))
    
    for i in list_of_images:
        if 'positive' in i:
            y.append(1)
        else:
            y.append(0)
      
            
            
    return x, y

In [ ]:
X, Y = prepare_data(MURA_train)
print(K.image_data_format())

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size=0.2, random_state=1)
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 16

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.Activation('relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
model.compile(optimizer = Adam(lr = 1e-3, beta_1=0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad = True),
              loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
train_generator = train_datagen.flow(np.array(X), Y, batch_size=batch_size)
validation_generator = val_datagen.flow(np.array(X), Y, batch_size=batch_size)

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
checkpoint = ModelCheckpoint('AlexNet-best.hdf5',
                monitor='val_binary_accuracy', verbose=1, save_best_only=True, 
                save_weights_only=True, mode='max', period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.001)

callbacks_list = [checkpoint, reduce_lr]


history = model.fit_generator(train_generator, steps_per_epoch=nb_train_samples // batch_size, 
                              epochs=70, callbacks=callbacks_list, 
                              validation_data=validation_generator, 
                              validation_steps=nb_validation_samples // batch_size)

In [ ]:
model.save('AlexNet.h5')

In [ ]:
from keras.callbacks import *
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12,6)

acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'red', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()

plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'red', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')

plt.legend()

plt.show()